<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/meti-94/OpenQA.git

Cloning into 'OpenQA'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 209 (delta 81), reused 129 (delta 28), pack-reused 0
Receiving objects: 100% (209/209), 102.55 MiB | 35.12 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [1]:
import pandas as pd
import sys
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('/content/OpenQA/validation.xlsx')

In [3]:
df.loc[1, 'answers']

"[('20e3dd8b-0f8e-4379-8552-58aef7ca3857', 'again chose', 1.0, 1.0, '0.90925', 114750)]"

In [3]:
X, y = [], []
for idx1, row in df.iterrows():
  try:
    temporary = eval(row['answers'])
  
    for idx2, candidate in enumerate(temporary):
      if len(candidate)==6:
        if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
          X.append((candidate[2], candidate[3], float(candidate[4])))
          y.append(1)
        else:
          X.append((candidate[2], candidate[3], float(candidate[4])))
          y.append(0)
      else:
        if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
          X.append((candidate[2], candidate[3], 1.0))
          y.append(1)
        else:
          X.append((candidate[2], candidate[3], 1.0))
          y.append(0)
  except:
    continue


In [4]:
len(X), len(y), sum(y), X[:4]

(223245,
 223245,
 16306,
 [(1.0, 1.0000000000000002, 0.92794),
  (1.0, 1.0, 0.90925),
  (1.0, 1.0, 0.9384100000000001),
  (1.0, 0.9467529114555084, 0.9338200000000001)])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
regr = MLPRegressor(hidden_layer_sizes=(3, 10, 10, ),
                    activation='tanh',
                    solver='adam',
                    learning_rate='adaptive',
                    ).fit(X_train, y_train)

In [7]:
regr.predict(X_test[:2])

array([ 0.01141934, -0.0119595 ])

In [9]:
regr.score(X_test, y_test)

0.26210440491094733

In [10]:
import pickle
with open('classifier.pkl', 'wb') as fid:
    pickle.dump(regr, fid)    